# BS Optimization

- Solving a constrained resource allocation problem using a sample Bank balance sheet and (mostly) realistic balance sheet constraints

### Background

Bank resource management has become significantly more complex in the last decade. After the 2008 financial crisis, a slew of new regulatory frameworks were enacted, complicating balance sheet management. It was no longer immediately obvious which products were more profitable or what the optimal balance sheet mix was for a given "type" of bank (i.e., Universal vs. Investment vs. Retail bank, etc).

This is where optimization can help. These constraints can be expressed as equations, and by creating bounds on the Balance Sheet (abbreviated here on out as B/S) line items, these equations can be solved in a way that maximizes the profitability of the balance sheet.

### Balance Sheet Overview

Sample data can be found here: https://github.com/marvelje/bs_optimization/tree/main/data

- For purposes of this analysis, I've created a fake bank balance sheet of ~40 line items. I've relied on my experience at a big bank to construct a reasonably realistic scenario for a "universal" bank (i.e., a bank offering a full suite of banking services across retail, commercial, and markets). Many of these line items will look familiar to anyone who's worked in finance. You'll notice auto loans, various types of mortgage products, commercial lending, markets, and a whole suite of deposit products.
- For any optimization to have meaningful results, the line items should be broken down in a way where the bundled products have similar levels of profitability and "resource footprints" (more on this last bit under the Constraints Overview). The more aggregated the data, the less useful it becomes. Therefore, I've broken down a given product category where a subgroup has meaningfully different characteristics. For example, I broke Auto lending down between "Prime" and "Subprime" since the returns and resource footprint is materially different.
- I then created spreads (profitability expressed in basis points, or bps), along with upper and lower bounds. I.e., how much will I allow the products to grow and shrink during the optimization.
- To start, I generated somewhat arbitrary numbers, ensuring only that Assets = Liabilities and the constraints are currently satisfied

### Constraint overview

- This analysis presents a simplified version of the variety of constraints that a large bank currently faces
- Risk weighted assets:
    - This framework attemnpts to turn a given balance into a Risk Weight equivalent. For example, a mortgage loan may have a Standardized RWA of 50%. This means that a 100 dollar mortgage will generate Standardized RWA of 50.
    - The regulation states that the bank must have equity equal to a fixed % of it's Risk Weighted Assets. This is the crux of the give and take of the optimization. Equity is expensive! The bank would prefer not to hold more equity than it needs to so understanding the trade-offs here is important.
    - There are two RWA frameworks: Advanced and Standardized. A given product may have two different weights across each of these frameworks
- GSIB:
    - This is not a standalone constraint, but rather adds to the RWA minimums described above.
    - For example, the RWA minimum may be 8% plus a GSIB surchage, let's say 3%. The GSIB surcharge scales up or down with changes to the balance sheet, with some products contributing more to the GSIB surcharge than others.
    - In reality, this is a "stair-step" framework. The GSIB surcharge moves in increments of 0.5% A given score may place you in the 3% "bucket" until you hit the subsequent threshold, bumping you up to 3.5%. To keep this a linear problem, I removed this stair step and interpolated the GSIB contribution. In othe words, a small increase in balances may make the surcharge 3.05%, when in reality we would've stayed in the 3% bucket. This is the correct approach under a linear framework, although this could be reframed as a non-linear problem to account for this 
- Leverage:
    - This works similarly, but is a bit more straightforward. For example, the leverage requirement for CET1 states that you must have enough Equity to cover at least a fied % of the total assets. Functions similarly as RWA, but there's no intermediate step of 
    
The TLAC stack:
- Each RWA constraint has four components: CET1, T1, Total Capital, and TLAC. Collectively, these comprise the "resource stack". Different resource can contribute to meeting the minimum requirements for each of the levels. And they are additive. For example, the CET1 requirement might be 11% of RWA and this can only be met through equity. T1 minimum might be 13% but can be met with either Equity or Preferred Equity. Total Capital may be 15%, but subordinated debt can be used, etc. Equity is the most expense resource so that will not be used to meet requirements beyond CET1 if there are other options.
- For Leverage, there are only requirements at the CET1 and T1 level.
- Below is an illustration of the typical bank resource stack

![CRE-The-Capital-Stack.png](attachment:CRE-The-Capital-Stack.png)

Source: https://www.crowdengine.com/cre-basics-the-capital-stack/

In [1]:
import pandas as pd
import numpy as np
import scipy.optimize as spo

In [2]:
# Set options to print all columns of the dataframe

pd.set_option('display.max_columns', None)

In [3]:
# Read in BS data

df = pd.read_excel('./data/sample_bs.xlsx', index_col=0, nrows=40, usecols='A:AO')

In [4]:
df.head()

,Product,start,grow,shrink,spread,A_L,b1_leverage,a_rwa,s_rwa,gsib_leverage,gsib_xjd_claim,gsib_xjd_liab,gsib_intrafin_claim,gsib_intrafin_liab,gsib_securities,gsib_payment,gsib_auc,gsib_underwriting,gsib_otc,`,gsib_level3,CET1_resource,T1_resource,total_capital_resource,TLAC_resource,gsib_leverage_score,gsib_xjd_claim_score,gsib_xjd_liab_score,gsib_intrafin_claim_score,gsib_intrafin_liab_score,gsib_securities_score,gsib_payment_score,gsib_auc_score,gsib_underwriting_score,gsib_otc_score,gsib_trading_score,gsib_level3_score,total_score,cet1_contr,cet1_contr_per_balance
Index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,prime_auto,15000,8000,-5000,80,1,1,0.2,1.0,1,0.0,0,0.0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,3.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,0.0,3.0,0.000144,9.631978e-09
2,subprime_auto,4000,2000,-1000,150,1,1,0.8,1.0,1,0.0,0,0.0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0.8,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,0.0,0.8,0.000039,9.631978e-09
3,mtg_30_fixed,50000,30000,-15000,70,1,1,0.3,0.5,1,0.0,0,0.0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,10.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,0.0,10.0,0.000482,9.631978e-09
4,mtg_15_fixed,15000,8000,-3000,72,1,1,0.3,0.5,1,0.0,0,0.0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,3.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,0.0,3.0,0.000144,9.631978e-09
5,mtg_7_fixed,5000,2500,-2000,70,1,1,0.3,0.5,1,0.0,0,0.0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,0.0,1.0,0.000048,9.631978e-09


In [5]:
# Read in constraints

constraints = pd.read_excel('./data/sample_bs.xlsx', index_col=0, sheet_name='constraints')

constraints

,CET1,T1,total_capital,TLAC
SRWA,0.080000,0.095,0.115,0.195
ARWA,0.080000,0.095,0.115,0.195
leverage,0.075000,0.095,NaN,NaN
GSIB,0.000048,NaN,NaN,NaN


## Objective function

- This is simply an array of the negative return (i.e., profitability)
- Since the optimization will actually be minimizing this, we just flip the sign

In [6]:
# Define NSI to be passed as part of args

spread = np.array(df['spread'])

In [7]:
def obj_fun(x, spread):
    profitability = 0
    
    for i in range(len(x)):
        profitability += -(spread[i] * x[i])
    
    return profitability

## Starting Guess

In [8]:
# Starting guess is the initial B/S balances
x0 = np.array(df['start'])

## Inequality Constraints

In [9]:
# Extract constraints from the inputs

sRWA_mins = list(constraints.loc['SRWA', :])
aRWA_mins = list(constraints.loc['ARWA', :])
lev_mins = list(constraints.loc['leverage',['CET1', 'T1']])

### Standardized RWA

In [10]:
# Standardized RWA: CET1
def srwa_cet1(x, df=df, sRWA_mins=sRWA_mins):
    # Define GSIB add-on
    gsib_addon = 0
    gsib_cont = np.array(df['cet1_contr_per_balance'])
    
    # Define base RWA min
    s_rwa = np.array(df['s_rwa'])
    
    # Define what counts as a resource towards solving the constraint
    cet1_resource = np.array(df['CET1_resource'])
    
    # Define the constraint threshold. The inequality has to be greater than 0.
    cet1_constraint = 0
    
    # Calculate GSIB add-on
    for i in range(len(x)):
        gsib_addon += x[i] * gsib_cont[i]
    
    # Define constrain
    for i in range(len(x)):
        cet1_constraint += -((sRWA_mins[0] + gsib_addon) * s_rwa[i] * x[i]) + cet1_resource[i] * x[i]
        
    return cet1_constraint

In [11]:
# Standardized RWA: T1
def srwa_t1(x, df=df, sRWA_mins=sRWA_mins):
    # Define GSIB add-on
    gsib_addon = 0
    gsib_cont = np.array(df['cet1_contr_per_balance'])
    
    # Define base RWA min
    s_rwa = np.array(df['s_rwa'])
    
    # Define what counts as a resource towards solving the constraint
    t1_resource = np.array(df['T1_resource'])
    
    # Define the constraint threshold. The inequality has to be greater than 0.
    t1_constraint = 0
    
    # Calculate GSIB add-on
    for i in range(len(x)):
        gsib_addon += x[i] * gsib_cont[i]
    
    # Define constrain
    for i in range(len(x)):
        t1_constraint += -((sRWA_mins[1] + gsib_addon) * s_rwa[i] * x[i]) + t1_resource[i] * x[i]
        
    return t1_constraint

In [12]:
# Standardized RWA: Total Capital
def srwa_tc(x, df=df, sRWA_mins=sRWA_mins):
    # Define GSIB add-on
    gsib_addon = 0
    gsib_cont = np.array(df['cet1_contr_per_balance'])
    
    # Define base RWA min
    s_rwa = np.array(df['s_rwa'])
    
    # Define what counts as a resource towards solving the constraint
    tc_resource = np.array(df['total_capital_resource'])
    
    # Define the constraint threshold. The inequality has to be greater than 0.
    tc_constraint = 0
    
    # Calculate GSIB add-on
    for i in range(len(x)):
        gsib_addon += x[i] * gsib_cont[i]
    
    # Define constrain
    for i in range(len(x)):
        tc_constraint += -((sRWA_mins[2] + gsib_addon) * s_rwa[i] * x[i]) + tc_resource[i] * x[i]
        
    return tc_constraint

In [13]:
# Standardized RWA: TLAC
def srwa_tlac(x, df=df, sRWA_mins=sRWA_mins):
    # Define GSIB add-on
    gsib_addon = 0
    gsib_cont = np.array(df['cet1_contr_per_balance'])
    
    # Define base RWA min
    s_rwa = np.array(df['s_rwa'])
    
    # Define what counts as a resource towards solving the constraint
    tlac_resource = np.array(df['TLAC_resource'])
    
    # Define the constraint threshold. The inequality has to be greater than 0.
    tlac_constraint = 0
    
    # Calculate GSIB add-on
    for i in range(len(x)):
        gsib_addon += x[i] * gsib_cont[i]
    
    # Define constrain
    for i in range(len(x)):
        tlac_constraint += -((sRWA_mins[3] + gsib_addon) * s_rwa[i] * x[i]) + tlac_resource[i] * x[i]
        
    return tlac_constraint

### Advanced RWA

In [14]:
# Advanced RWA: CET1
def arwa_cet1(x, df=df, aRWA_mins=aRWA_mins):
    # Define GSIB add-on
    gsib_addon = 0
    gsib_cont = np.array(df['cet1_contr_per_balance'])
    
    # Define base RWA min
#     aRWA_mins = list(constraints.loc['ARWA', :])
    a_rwa = np.array(df['a_rwa'])
    
    # Define what counts as a resource towards solving the constraint
    cet1_resource = np.array(df['CET1_resource'])
    
    # Define the constraint threshold. The inequality has to be greater than 0.
    cet1_constraint = 0
    
    # Calculate GSIB add-on
    for i in range(len(x)):
        gsib_addon += x[i] * gsib_cont[i]
    
    # Define constrain
    for i in range(len(x)):
        cet1_constraint += -((aRWA_mins[0] + gsib_addon) * a_rwa[i] * x[i]) + cet1_resource[i] * x[i]
        
    return cet1_constraint

In [15]:
# Advanced RWA: T1
def arwa_t1(x, df=df, aRWA_mins=aRWA_mins):
    # Define GSIB add-on
    gsib_addon = 0
    gsib_cont = np.array(df['cet1_contr_per_balance'])
    
    # Define base RWA min
    a_rwa = np.array(df['a_rwa'])
    
    # Define what counts as a resource towards solving the constraint
    t1_resource = np.array(df['T1_resource'])
    
    # Define the constraint threshold. The inequality has to be greater than 0.
    t1_constraint = 0
    
    # Calculate GSIB add-on
    for i in range(len(x)):
        gsib_addon += x[i] * gsib_cont[i]
    
    # Define constrain
    for i in range(len(x)):
        t1_constraint += -((aRWA_mins[1] + gsib_addon) * a_rwa[i] * x[i]) + t1_resource[i] * x[i]
        
    return t1_constraint

In [16]:
# Advanced RWA: Total Capital
def arwa_tc(x, df=df, aRWA_mins=aRWA_mins):
    # Define GSIB add-on
    gsib_addon = 0
    gsib_cont = np.array(df['cet1_contr_per_balance'])
    
    # Define base RWA min
    a_rwa = np.array(df['a_rwa'])
    
    # Define what counts as a resource towards solving the constraint
    tc_resource = np.array(df['total_capital_resource'])
    
    # Define the constraint threshold. The inequality has to be greater than 0.
    tc_constraint = 0
    
    # Calculate GSIB add-on
    for i in range(len(x)):
        gsib_addon += x[i] * gsib_cont[i]
    
    # Define constrain
    for i in range(len(x)):
        tc_constraint += -((aRWA_mins[2] + gsib_addon) * a_rwa[i] * x[i]) + tc_resource[i] * x[i]
        
    return tc_constraint

In [17]:
# Advanced RWA: TLAC
def arwa_tlac(x, df=df, aRWA_mins=aRWA_mins):
    # Define GSIB add-on
    gsib_addon = 0
    gsib_cont = np.array(df['cet1_contr_per_balance'])
    
    # Define base RWA min
    a_rwa = np.array(df['a_rwa'])
    
    # Define what counts as a resource towards solving the constraint
    tlac_resource = np.array(df['TLAC_resource'])
    
    # Define the constraint threshold. The inequality has to be greater than 0.
    tlac_constraint = 0
    
    # Calculate GSIB add-on
    for i in range(len(x)):
        gsib_addon += x[i] * gsib_cont[i]
    
    # Define constrain
    for i in range(len(x)):
        tlac_constraint += -((aRWA_mins[3] + gsib_addon) * a_rwa[i] * x[i]) + tlac_resource[i] * x[i]
        
    return tlac_constraint

### Leverage

In [18]:
# Leverage: CET1
def lev_cet1(x, df=df, lev_mins=lev_mins):
    
    # Define base leverage contribution
    b1_lev = np.array(df['b1_leverage'])
    
    # Define what counts as a resource towards solving the constraint
    cet1_resource = np.array(df['CET1_resource'])
    
    # Define the constraint threshold. The inequality has to be greater than 0.
    cet1_constraint = 0
    
    # Define constrain
    for i in range(len(x)):
        cet1_constraint += -(lev_mins[0] * b1_lev[i] * x[i]) + cet1_resource[i] * x[i]
        
    return cet1_constraint

In [19]:
# Leverage: T1
def lev_t1(x, df=df, lev_mins=lev_mins):
    
    # Define base leverage contribution
    b1_lev = np.array(df['b1_leverage'])
    
    # Define what counts as a resource towards solving the constraint
    t1_resource = np.array(df['T1_resource'])
    
    # Define the constraint threshold. The inequality has to be greater than 0.
    t1_constraint = 0
    
    # Define constrain
    for i in range(len(x)):
        t1_constraint += -(lev_mins[1] * b1_lev[i] * x[i]) + t1_resource[i] * x[i]
        
    return t1_constraint

## Equality Constraints

- Assets must equal liabilities

In [20]:
def eq_constraint(x, df=df):   
    # Create array of Asset / Liability weights
    a_l_weight = np.array(df['A_L'])
    
    # Define target (assets must equal liabilities so the difference is 0)
    eq = 0
    
    for i in range(len(x)):
        eq = eq - x[i] * a_l_weight[i]
    
    return eq

## Consolidate Constraints

Convert to Scipy minimize formatting

In [21]:
con1 = {'type': 'ineq', 'fun': srwa_cet1}
con2 = {'type': 'ineq', 'fun': srwa_t1}
con3 = {'type': 'ineq', 'fun': srwa_tc}
con4 = {'type': 'ineq', 'fun': srwa_tlac}
con5 = {'type': 'ineq', 'fun': arwa_cet1}
con6 = {'type': 'ineq', 'fun': arwa_t1}
con7 = {'type': 'ineq', 'fun': arwa_tc}
con8 = {'type': 'ineq', 'fun': arwa_tlac}
con9 = {'type': 'ineq', 'fun': lev_cet1}
con10 = {'type': 'ineq', 'fun': lev_t1}
con11 = {'type': 'eq', 'fun': eq_constraint}

cons = [con1, con2, con3, con4, con5, con6, con7, con8, con9, con10, con11]

## Upper and Lower Bounds

In [22]:
# Create upper and lower bounds. This is the grow and shrink amount for each item

df['u_bound'] = df['start'] + df['grow']
df['l_bound'] = df['start'] + df['shrink']

# Create tuple of lower / upper bounds
df['bounds'] = df.apply(lambda row: tuple((row['l_bound'], row['u_bound'])), axis=1)

In [23]:
# Define bounds in format needed for minimize

bounds = tuple(df['bounds'])

## Minimize

Solvers:
- trust-constr
- SLSQP

In [24]:
sol = spo.minimize(obj_fun, x0, method = 'trust-constr', bounds=bounds, constraints=cons, args = (spread))

/Users/jeffreymarvel/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


In [25]:
sol.success

True

In [26]:
sol.fun

-145061854.78840116

In [27]:
sol.x

array([ 1.00000000e+04,  6.00000000e+03,  8.00000000e+04,  2.30000000e+04,
        7.50000000e+03,  3.00000000e+04,  1.35000000e+04,  8.00000000e+04,
        1.75000000e+04,  3.21667725e+04,  4.65000000e+04,  2.80000000e+04,
        8.50000000e+04,  1.05000000e+05,  7.50000000e+04,  9.50000000e+04,
        3.40000000e+04,  1.30000000e+04,  1.50000000e+03,  1.23769098e-16,
        2.40000000e+05,  8.85000000e+04,  1.10000000e+05,  2.60000000e+04,
        1.20000000e+04,  3.50000000e+03,  7.50000000e+04,  3.20000000e+04,
        2.20000000e+04,  1.25000000e+04,  1.60000000e+04,  9.50000000e+03,
        5.00000000e+03,  2.00000000e+03,  3.00000000e+03, -3.98440647e-17,
        6.31113597e+04,  1.12419837e+04,  7.79662017e+03,  4.35168089e+04])

In [28]:
test = pd.DataFrame(sol.x)

In [29]:
test.to_excel('./results/scr.xlsx')